In [1]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib qt

nx = 9
ny = 6

In [2]:
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

In [3]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners 
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        #img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(1)


ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# print("mtx: " ,mtx)
# print("dist: " ,dist)

# Read in an image
img = cv2.imread('camera_cal/calibration1.jpg')
undistorted = cal_undistort(img, objpoints, imgpoints)

cv2.imshow('img',undistorted)
cv2.waitKey(1000)
print("undistorted")
#cv2.destroyAllWindows()


undistorted


In [4]:
def corners_unwarp(undistorted_img, mtx, dist):
    # c) define 4 destination points dst = np.float32([[,],[,],[,],[,]])
    #dst = np.float32([[0,0],[undistorted_img.shape[1]-100,100],[undistorted_img.shape[1]-100,undistorted_img.shape[0]-100],[100,undistorted_img.shape[0]-100]])

    print("img.shape[0]", undistorted_img.shape[0])
    print("img.shape[1]", undistorted_img.shape[1])
#    src  =  np.float32([[undistorted_img.shape[1]//2 - 20, undistorted_img.shape[0]//2], [undistorted_img.shape[1]//2+20, undistorted_img.shape[0]//2], [undistorted_img.shape[1], undistorted_img.shape[0]], [0, undistorted_img.shape[0]]])
#    src  =  np.float32([[600, undistorted_img.shape[0]//2 + 65], [undistorted_img.shape[1]-600, undistorted_img.shape[0]//2+65], [undistorted_img.shape[1], undistorted_img.shape[0]], [0, undistorted_img.shape[0]]])
    src  =  np.float32([[500, undistorted_img.shape[0]//2 + 75],                             [undistorted_img.shape[1] - 500, undistorted_img.shape[0]//2 + 75],                                 [undistorted_img.shape[1], undistorted_img.shape[0]], [0, undistorted_img.shape[0]]])
    dst  =  np.float32([[0, 0],                                                          [undistorted_img.shape[1], 0],                                 [undistorted_img.shape[1]-200, undistorted_img.shape[0]], [200, undistorted_img.shape[0]]])
#    dst  =  np.float32([[undistorted_img.shape[1]//2 -20, undistorted_img.shape[0]//2], [undistorted_img.shape[1]//2+20, undistorted_img.shape[0]//2], [undistorted_img.shape[1], undistorted_img.shape[0]], [0, undistorted_img.shape[0]]])    

    
    # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    print("M: ", M)
    # e) use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(undistorted_img, M, (undistorted_img.shape[1], undistorted_img.shape[0]), flags=cv2.INTER_LINEAR)
    
    return warped, M

In [5]:
def hls_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    image_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    channel_s = image_hls[:,:,2]
    # 3) Return a binary image of threshold result
    binary = np.zeros_like(channel_s)
    binary[(channel_s > thresh[0]) & (channel_s <= thresh[1])] = 1
    binary_output = binary # placeholder line
    return binary_output
    


In [19]:
img = cv2.imread('test_images/straight_lines1.jpg')
undistorted_img = cal_undistort(img, objpoints, imgpoints)
cv2.imshow('img_orig',undistorted_img)
cv2.waitKey(1000)

#rgb = img[...,::-1].copy()
hls_binary = hls_select(undistorted_img, thresh=(70, 255))
hls_binary = hls_binary*255
cv2.imshow('img_binary', hls_binary)
cv2.waitKey(1000)

top_down, perspective_M = corners_unwarp(hls_binary, mtx, dist)
cv2.imshow('img_unwarped',top_down)
cv2.waitKey(1000)

img.shape[0] 720
img.shape[1] 1280
M:  [[-5.99713056e-01 -1.66427547e+00  1.02381636e+03]
 [ 0.00000000e+00 -2.20373027e+00  9.58622669e+02]
 [-0.00000000e+00 -2.60043042e-03  1.00000000e+00]]


-1